In [18]:
%run PVModel_functions.ipynb

In [19]:
%run PVsystem_filterfunction_jack-may.ipynb

In [20]:
import pandas as pd
import numpy as np
from datetime import datetime
import datatools
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
import pvlib

%matplotlib inline 
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

mpl.rcParams['font.size']=12
mpl.rcParams['lines.linewidth']=1
mpl.rcParams['xtick.labelsize']=10
#mpl.rcParams['font.weight']='bold'
mpl.rcParams['axes.titlesize']=22

tablename = 'dbo.SNLA_MET_DataOut'

In [21]:
engine = datatools.database.create_mss_engine()

NameError: name 'met' is not defined

In [22]:
#Import 1-min Weather Data
start = '2020-01-01 00:00:00'
#end = pd.to_datetime('today')
end = '2021-01-01 00:00:00'
met_sql = f"select * from {tablename} where TmStamp between '{start}' and '{end}';"
#met_sql = f"select * from {tablename}"
met = pd.read_sql(met_sql, engine, index_col='TmStamp')
met.index = pd.DatetimeIndex(met.index)
met.index = met.index.tz_localize('MST')

In [34]:
#Organizing all necessary data into one df to easily filter & export

meteo = met['Global_Wm2_Avg'].to_frame()
meteo.rename(columns = {'Global_Wm2_Avg' : 'GHI'}, inplace = True)
meteo['DNI'] = met['Direct_Wm2_Avg']
meteo['DHI'] = met['Diffuse_Wm2_Avg']
meteo['Wind Speed'] = met['WS_ms_Mean']
meteo['Ambient Temp'] = met['Temp_C_Avg']
meteo['Relative Humidity'] = met['RH_pct_Avg']

In [35]:
#Filtering data based on physical bounds

meteo = simple_filter(meteo, 'GHI', lower=0, upper=1300, apply=True)
meteo = simple_filter(meteo, 'DNI', lower=0, upper=1200, apply=True)
meteo = simple_filter(meteo, 'DHI', lower=0, upper=800, apply=True)
meteo = simple_filter(meteo, 'Wind Speed', lower=0, upper=32, apply=True)
meteo = simple_filter(meteo, 'Ambient Temp', lower=-40, upper=60, apply=True)
meteo = simple_filter(meteo, 'Relative Humidity', lower=0, upper=100, apply=True)

In [36]:
#Resample into hourly increments
meteo = meteo.resample('H', closed='right', label='right').mean()

In [37]:
#Count NaN's to see percent filtered
print("The percent of filtered data is " + str(100*((meteo.isna().sum()[1])/8785)))

The percent of filtered data is 46.22652248150256


Albedo comes from the sys files, is filtered from 0.15<x<0.25 and is resampled on monthly intervals using rhs closed & label methods

In [38]:
meteo.to_csv('albq_raw_meteo.csv')